# Stepper Motor Control via I2C


## Particle CLI

### Installation

In [1]:
%%capture
!bash <( curl -sL https://particle.io/install-cli )

# path to the particle cli. May be environment dependent.
particle_cli = "/root/bin/particle"

### Utility functions

In [2]:
import re
import subprocess

# regular expression to strip ansi control characters
ansi = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# decode byte string and strip ansi control characters
def decode_bytes(byte_string):
    if isinstance(byte_string, bytes):
        result = byte_string.decode("utf-8")
    return ansi.sub("", result)

# streamline call to the particle-cli
def particle(args):
    process = subprocess.run(["/root/bin/particle"] + args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
    process.stdout = decode_bytes(process.stdout)
    process.stderr = decode_bytes(process.stderr)
    return process

### Login to Particle

In [ ]:
import getpass

# prompt for username and password
username = getpass.getpass(prompt="Username: ")
password = getpass.getpass(prompt="Password: ")

# attempt login
output = particle(["login", "--username", username, "--password", password])

# report results
if output.returncode:
    print(f"Return code = {output.returncode}")
    print(output.stderr)
else:
    print(output.stdout)

Username: ··········
Password: ··········
> Successfully completed login!



### Select a device

The following cell downloads a list of all user devices and creates a list of device names. Here we choose the first name in the list for the rest of this notebook. If this is not the device to be used, then modify this cell accordingly.

In [ ]:
devices = [line.split()[0] for line in particle(["list"]).stdout.splitlines()]
device_name = devices[0]
print(particle(["list", device_name]).stdout)

jck_argon_01 [e00fce68eaceb1faa7cf7193] (Argon) is online



## Project: Motor Control



### Grove I2C Motor Driver V1.3

[SeeedStudio Documentation](https://wiki.seeedstudio.com/Grove-I2C_Motor_Driver_V1.3/)

[Github repository](https://github.com/Seeed-Studio/Grove_I2C_Motor_Driver_v1_3)

![](https://files.seeedstudio.com/wiki/Grove-I2C_Motor_Driver_V1.3/img/I2CMotorDriver-1.jpg)

Note the default address ``0x0f``.

** It turns out this motor driver requires a 5 volt logic. The Particle Argon is capable of 3.3v only, thus not electrically compatible. This is confirmed by the absence of a code library supporting this motor driver on Particle Argon.**

**New motor drivers are on order.**

## Prototype

### Create Project

In [ ]:
print(particle(["project", "create", "--name", "myproject", "."]).stdout)

Initializing project in directory myproject...
> A new project has been initialized in directory myproject



### Change working directory

The Particle CLI assumes one is working in the top  project directory.

In [ ]:
%cd myproject

/content/myproject


### Add relevant libraries



In [ ]:
print(particle(["library", "search", "motor"]).stdout)

> Found 6 libraries matching motor
Adafruit-MotorShield-V2 1.0.0 12330 A library for the Adafruit Motor Shield V2 for Particle devices
Serial_Controlled_Motor_Driver 1.0.3 814 Use this to command the SCMD and connected chain.
SeeedMotorShield 0.0.1 572 Library for controling the DC Motor shield by Seeed
StepMotorController 0.0.32 479 Step Motor Controller Interface     
DualMotor 0.0.2 215 Custom DualMotor Library
PatriotMotorized 2.0.0 16 Extend Patriot IoT to support motorized devices.



In [ ]:
print(particle(["library", "add", "Grove_I2C_Motor_Driver_v1_3"]).stdout)

Library Grove_I2C_Motor_Driver_v1_3 not found



### Create source file

In [ ]:
%%writefile src/myproject.ino


Overwriting src/myproject.ino


### Compiling

In [ ]:
print(particle(["compile", "argon", "--saveTo", "myproject.bin"]).stdout)


Compiling code for argon

Including:
    src/myproject.ino
    project.properties

attempting to compile firmware
downloading binary from: /v1/binaries/5f91c30f9c09c651a428aa51
saving to: myproject.bin
Memory use:
   text	   data	    bss	    dec	    hex	filename
   6588	    108	   1112	   7808	   1e80	/workspace/target/workspace.elf

Compile succeeded.
Saved firmware to: /content/myproject/myproject.bin



### Flash firmware

In [ ]:
print(particle(["flash", device_name, "myproject.bin"]).stdout)

Including:
    myproject.bin

attempting to flash firmware to your device jck_argon_01
Flash device OK: Update started

Flash success!

